# Post Processing

This notebook demonstrates how to use `history.postprocessing` module. For more details on the Post-Processing [README](README.md). You can also use the prefect version by using `history.postprocessing_prefect` module.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import history.postprocessing as pp
from pathlib import Path
import pandas as pd
import geoutils as gu

/home/godinlu/micromamba/envs/history/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ⚙️ Settings and Path Management

Main directories used in the post-processing pipeline:

- **`raw_dir`** → Contains all raw input archives.  
- **`extracted_dir`** → Stores extracted archives.  
- **`processing_dir`** → Main workspace for intermediate and processed data.  
- **`plots_dir`** → Output folder for generated plots and visualizations.

> 💡 These directories are automatically created when running the Prefect flows.


In [4]:
proc_dir = Path("/mnt/summer/USERS/DEHECQA/history/output/processing")

OVERWRITE = False
DRY_RUN = False # set this to True to avoid process
PDAL_EXEC_PATH = "/home/godinlu/micromamba/envs/pdal/bin/pdal"
MAX_WORKERS = 4

### 🧩 Step 1 — Extract archives

In [ ]:
raw_dir = "/mnt/summer/USERS/DEHECQA/history/output/raw"
extracted_dir = "/mnt/summer/USERS/DEHECQA/history/output/extracted"

pp.pipeline.uncompress_all_submissions(raw_dir, extracted_dir, OVERWRITE, MAX_WORKERS)

### 🧩 Step 2 — Analyse submissions and create symlinks

In [ ]:
extracted_dir = "/mnt/summer/USERS/DEHECQA/history/output/extracted"
symlinks_dir = proc_dir / "symlinks"

pp.pipeline.index_submissions_and_link_files(extracted_dir, symlinks_dir)

In [ ]:
symlinks_directories = list((proc_dir / "symlinks").iterdir())
pp.viz.visualize_files_presence_map(symlinks_directories)

### 🧩 Step 3 — Create auxiliary data symlinks

In [8]:
references_data_mapping = {
    ("casa_grande", "aerial") : {
        "ref_dem": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_zoom_5m.tif",
        "ref_dem_mask": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_zoom_5m_mask.tif",
        "landcover": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_landcover_zoom.tif"
    },
    ("casa_grande", "kh9mc") : {
        "ref_dem": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_large.tif",
        "ref_dem_mask": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_ref_dem_large_mask.tif",
        "landcover": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/casagrande_landcover_large.tif"
    },
    ("iceland", "aerial") : {
        "ref_dem": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_zoom_5m.tif",
        "ref_dem_mask": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_zoom_5m_mask.tif",
        "landcover": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_landcover_zoom.tif"
    },
    ("iceland", "kh9mc") : {
        "ref_dem": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_large.tif",
        "ref_dem_mask": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_ref_dem_large_mask.tif",
        "landcover": "/mnt/summer/USERS/DEHECQA/history/output/aux_data/iceland_landcover_large.tif"
    }
}

# Same references dem for KH-9 MC and KH-9 PC
references_data_mapping[("casa_grande", "kh9pc")] = references_data_mapping[("casa_grande", "kh9mc")]
references_data_mapping[("iceland", "kh9pc")] = references_data_mapping[("iceland", "kh9mc")]


references_data = pp.ReferencesData(references_data_mapping)

### 🧩 Step 4 — Convert point clouds to DEMs

In [ ]:
dense_pointclouds_dir = proc_dir / "symlinks" / "dense_pointclouds"
generated_raw_dems_dir = proc_dir / "generated_raw_dems"
pointcloud_files = list(dense_pointclouds_dir.iterdir())

pp.pipeline.process_pointclouds_to_dems(
    pointcloud_files,
    generated_raw_dems_dir,
    references_data,
    PDAL_EXEC_PATH,
    OVERWRITE,
    DRY_RUN,
    MAX_WORKERS
)

In [ ]:
dem_files = [
    "/mnt/summer/USERS/DEHECQA/history/output/processing/symlinks/dems/CoSP_CG_PC_PP_CY_GN_PN_MN_dem.tif",
    "/mnt/summer/USERS/DEHECQA/history/output/processing/symlinks/dems/CoSP_IL_PC_PP_CY_GN_PN_MN_dem.tif"
]
provided_raw_dems_dir = proc_dir / "provided_raw_dems"

pp.pipeline.add_provided_dems(dem_files, provided_raw_dems_dir, references_data)

### 🧩 Step 5 — Coregister DEMs

In [ ]:
generated_raw_dems_dir = proc_dir / "generated_raw_dems"
provided_raw_dems_dir = proc_dir / "provided_raw_dems"
coregisterd_dems_dir = proc_dir / "coregistered_dems"

dem_files = list(generated_raw_dems_dir.glob("*-DEM.tif")) + list(provided_raw_dems_dir.glob("*-DEM.tif"))


pp.pipeline.coregister_dems(dem_files, coregisterd_dems_dir, references_data, OVERWRITE, 1)

In [ ]:
symlinks_directories = list((proc_dir / "symlinks").iterdir())
for dirname in ["generated_raw_dems", "provided_raw_dems", "coregistered_dems"]:
    symlinks_directories.append(proc_dir / dirname)
pp.viz.visualize_files_presence_map(symlinks_directories)

### 🧩 Step 6 — Generate Differential DEMs (DDEMs)


In [ ]:
generated_raw_dems_dir = proc_dir / "generated_raw_dems"
provided_raw_dems_dir = proc_dir / "provided_raw_dems"
coregisterd_dems_dir = proc_dir / "coregistered_dems"

before_coreg_ddems_dir = proc_dir / "ddems" / "before_coregistration"
after_coreg_ddems_dir = proc_dir / "ddems" / "after_coregistration"

before_coreg_files = list(generated_raw_dems_dir.glob("*-DEM.tif")) + list(provided_raw_dems_dir.glob("*-DEM.tif"))
after_coreg_files = list(coregisterd_dems_dir.glob("*-DEM.tif"))

pp.pipeline.generate_ddems(before_coreg_files, before_coreg_ddems_dir, references_data, OVERWRITE, MAX_WORKERS)
pp.pipeline.generate_ddems(after_coreg_files, after_coreg_ddems_dir, references_data, OVERWRITE, MAX_WORKERS)

### 🧩 Step 7 — Compute general statistics

In [6]:
nmad_multiplier: float = 3.0

generated_raw_dems_dir = proc_dir / "generated_raw_dems"
provided_raw_dems_dir = proc_dir / "provided_raw_dems"
dem_files = list(generated_raw_dems_dir.glob("*-DEM.tif")) + list(provided_raw_dems_dir.glob("*-DEM.tif"))
df1 = pp.stats.compute_dems_statistics_df(dem_files, "raw_dem_")

coregisterd_dems_dir = proc_dir / "coregistered_dems"
df2 = pp.stats.compute_dems_statistics_df(list(coregisterd_dems_dir.glob("*-DEM.tif")), "coreg_dem_")

before_coreg_ddems_dir = proc_dir / "ddems" / "before_coregistration"
df3 = pp.stats.compute_dems_statistics_df(list(before_coreg_ddems_dir.glob("*-DDEM.tif")), "ddem_before_")

after_coreg_ddems_dir = proc_dir / "ddems" / "after_coregistration"
df4 = pp.stats.compute_dems_statistics_df(list(after_coreg_ddems_dir.glob("*-DDEM.tif")), "ddem_after_")

dense_pointclouds_dir = proc_dir / "symlinks" / "dense_pointclouds"
df5 = pp.stats.compute_pcs_statistics_df(list(dense_pointclouds_dir.iterdir()), "dense_pointcloud_")

sparse_pointclouds_dir = proc_dir / "symlinks" / "sparse_pointclouds"
df6 = pp.stats.compute_pcs_statistics_df(list(sparse_pointclouds_dir.iterdir()), "sparse_pointcloud_")

df7 = pp.stats.get_coregistration_statistics_df(list(coregisterd_dems_dir.glob("*-DEM.tif")))

combined_df = pd.concat([df1, df2, df3, df4, df5, df6, df7]).groupby(level=0).first()

combined_df.to_csv(proc_dir / "global_statistics.csv")

In [ ]:
def compute_inliers(nmad):
    nmad_dropped = nmad.dropna()
    threshold = nmad_dropped.median() + nmad_multiplier * gu.stats.nmad(nmad_dropped)
    return nmad <= threshold 
   
df = pd.read_csv(proc_dir / "global_statistics.csv", index_col="code")
inliers_df = df.groupby(["site", "dataset"], group_keys=False)[["ddem_after_nmad"]].apply(compute_inliers)
inliers_df = inliers_df.rename(columns={"ddem_after_nmad": "inliers"})
inliers_df.to_csv(proc_dir / "inliers.csv")

### 🧩 Step 8 — Compute landcover-based statistics

In [ ]:
after_coreg_ddems_dir = proc_dir / "ddems" / "after_coregistration"

landcover_df = pp.stats.compute_landcover_statistics(list(after_coreg_ddems_dir.glob("*-DDEM.tif")), references_data)
landcover_df.to_csv(proc_dir / "landcover_statistics.csv", index=None)

### 🧩 Step 9 — Generate STD DEMs

In [ ]:
df: pd.DataFrame = pd.read_csv(proc_dir / "global_statistics.csv", index_col="code")
inliers = pd.read_csv(proc_dir / "inliers.csv", index_col="code")["inliers"]

std_dems_dir = proc_dir / "std_dems"

for (site, dataset), group in df.groupby(["site", "dataset"]):
    group_inliers = group.loc[inliers]

    dem_files = group["coreg_dem_file"].dropna().to_list()
    dem_files_inliers = group_inliers["coreg_dem_file"].dropna().to_list() 

    pp.pipeline.generate_std_dem(dem_files, std_dems_dir / f"std_dem_{site}_{dataset}.tif", OVERWRITE)
    pp.pipeline.generate_std_dem(dem_files_inliers, std_dems_dir / f"std_dem_{site}_{dataset}_inliers.tif", OVERWRITE)

### 🧩 Step 10 — Compute landcover-based statistics on STD DEMs

In [ ]:
std_dems_dir = proc_dir / "std_dems"


std_lc_df = pp.stats.compute_landcover_statistics_on_std_dems(std_dems_dir.glob("*.tif"), references_data)
std_lc_df.to_csv(proc_dir / "std_landcover_statistics.csv", index=None)

### 🧩 Step 11 — Generate visualizations

In [ ]:
plot_dir = Path("/mnt/summer/USERS/DEHECQA/history/output/plots")

df = pd.read_csv(proc_dir / "global_statistics.csv", index_col = "code")
lc_df = pd.read_csv(proc_dir / "landcover_statistics.csv")
std_lc_df = pd.read_csv(proc_dir / "std_landcover_statistics.csv")

inliers = pd.read_csv(proc_dir / "inliers.csv", index_col = "code")["inliers"]



In [ ]:
pp.viz.barplot_var(
    df,
    plot_dir / "pointcloud_point_count.png",
    "dense_pointcloud_point_count",
    "Point count in dense point-cloud file",
)
pp.viz.barplot_var(
    df,
    plot_dir / "nmad_after_coregistration.png",
    "ddem_after_nmad",
    "NMAD of Altitude differences with ref DEM after coregistration by code",
)
pp.viz.barplot_var(
    df, plot_dir / "raw_dem_voids.png", "raw_dem_percent_nodata", "Raw DEM nodata percent"
)
pp.viz.generate_landcover_grouped_boxplot_from_std_dems(
    std_lc_df, plot_dir / "landcover_boxplot_from_std_dems.png"
)

std_lc_df_inliers = std_lc_df.loc[std_lc_df["code"].str.contains("_inliers")]
pp.viz.generate_landcover_grouped_boxplot_from_std_dems(
    std_lc_df_inliers, plot_dir / "landcover_boxplot_from_std_dems_inliers.png"
)

In [ ]:
for (site, dataset), group in df.groupby(["site", "dataset"]):
    # set the sub dir with site and dataset
    sub_dir = plot_dir / f"{site}_{dataset}"

    # add site and dataset before all title plots
    title_prefix = f"({site} - {dataset})"
    pp.viz.generate_plot_nmad_before_vs_after(
        group,
        sub_dir / "nmad_before_vs_after_coregistration.png",
        title=f"{title_prefix} NMAD of DEM differences before vs after coregistration",
    )
    pp.viz.generate_plot_coreg_shifts(
        group,
        sub_dir / "coregistration_shifts.png",
        title=f"{title_prefix} Coregistration shifts",
    )

    # Generate the same plots but with only inliers to increase the visibility
    group_inliers = group.loc[inliers]

    pp.viz.generate_plot_nmad_before_vs_after(
        group_inliers,
        sub_dir / "nmad_before_vs_after_coregistration_inliers.png",
        title=f"{title_prefix} NMAD of DEM differences before vs after coregistration (inliers only)",
    )
    pp.viz.generate_plot_coreg_shifts(
        group_inliers,
        sub_dir / "coregistration_shifts_inliers.png",
        title=f"{title_prefix} Coregistration shifts (inliers only)",
    )


In [ ]:
for (site, dataset), group in lc_df.groupby(["site", "dataset"]):
    # set the sub dir with site and dataset
    sub_dir = plot_dir / f"{site}_{dataset}"

    pp.viz.generate_landcover_grouped_boxplot(
        group,
        sub_dir / "landcover_grouped_boxplot.png",
        title=f"{title_prefix} Boxplot of Altitude difference with ref DEM by code/landcover",
    )
    # viz.generate_landcover_boxplot(lc_stats, sub_dir / "landcover_boxplot.png")
    pp.viz.generate_landcover_nmad(
        group,
        sub_dir / "landcover_nmad.png",
    title=f"{title_prefix} NMAD of Altitude difference with ref DEM by code/landcover",
    )

    # landcover plots inliers
    group_inliers = group[group["code"].isin(inliers.index[inliers])]
    pp.viz.generate_landcover_grouped_boxplot(
        group_inliers,
        sub_dir / "landcover_grouped_boxplot_inliers.png",
        title=f"{title_prefix} Boxplot of Altitude difference with ref DEM by code/landcover (inliers only)",
    )
    pp.viz.generate_landcover_nmad(
        group_inliers,
        sub_dir / "landcover_nmad_inliers.png",
        title=f"{title_prefix} NMAD of Altitude difference with ref DEM by code/landcover (inliers only)",
    )

In [ ]:
for (std_dem_file, site, dataset), _ in std_lc_df.groupby(["std_dem_file", "site", "dataset"]):
    output_path = plot_dir / f"{site}_{dataset}" / Path(std_dem_file).with_suffix(".png").name
    pp.viz.generate_std_dem_plots(std_dem_file, output_path)


In [ ]:
for (site, dataset), group in df.groupby(["site", "dataset"]):
    pp.iz.generate_coregistration_individual_plots(group, plot_dir / f"{site}_{dataset}" / "coregistrations")

In [ ]:
pp.viz.generate_all_mosaics(df, plot_dir, MAX_WORKERS)